# VacationPy
---

## Starter Code to Import Libraries and Load the Weather and Coordinates Data

In [3]:
# Dependencies and Setup
import hvplot.pandas
import pandas as pd
import requests

# Import API key
from api_keys import geoapify_key

In [4]:
# Load the CSV file created in Part 1 into a Pandas DataFrame
city_data_df = pd.read_csv("output_data/cities.csv")

# Display sample data
city_data_df.head()

,City_ID,City,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Country,Date
0,0,faya,18.3851,42.4509,22.06,35,21,2.60,SA,1666108228
1,1,farsund,58.0948,6.8047,13.30,100,0,7.65,NO,1666108228
2,2,new norfolk,-42.7826,147.0587,11.72,58,12,1.34,AU,1666108230
3,3,jamestown,42.0970,-79.2353,5.77,77,100,9.77,US,1666107934
4,4,lanzhou,36.0564,103.7922,14.53,48,59,1.20,CN,1666108230


---

### Step 1: Create a map that displays a point for every city in the `city_data_df` DataFrame. The size of the point should be the humidity in each city.

In [5]:
%%capture --no-display

# Configure the map plot
map_plot = city_data_df.hvplot.points('Lng',
            'Lat',
            geo=True, 
            tiles = 'OSM', 
            frame_width = 800, 
            frame_height = 500, 
            color='City',
            alpha=0.7,
            hover=True,
            hover_cols='all',
            use_index=False,
            s ='Humidity',
            scale= 1.2)

# Display the map
map_plot

:Overlay
   .WMTS.I   :WMTS   [Longitude,Latitude]
   .Points.I :Points   [Lng,Lat]   (City,Humidity,City_ID,Max Temp,Cloudiness,Wind Speed,Country,Date)

### Step 2: Narrow down the `city_data_df` DataFrame to find your ideal weather condition

In [28]:
# Narrow down cities that fit criteria and drop any results with null values
criteria_df = city_data_df.loc[(city_data_df['Max Temp'] > 5) & (city_data_df['Max Temp'] < 25) & (city_data_df['Humidity'] < 75) & (city_data_df['Wind Speed'] < 7)]


# Drop any rows with null values
criteria_df.dropna()

# Display sample data
criteria_df

,City_ID,City,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Country,Date
0,0,faya,18.3851,42.4509,22.06,35,21,2.60,SA,1666108228
2,2,new norfolk,-42.7826,147.0587,11.72,58,12,1.34,AU,1666108230
4,4,lanzhou,36.0564,103.7922,14.53,48,59,1.20,CN,1666108230
6,6,albany,42.6001,-73.9662,10.36,74,29,2.10,US,1666108231
9,9,saint-pierre,-21.3393,55.4781,22.82,68,0,5.14,RE,1666108236
...,...,...,...,...,...,...,...,...,...,...
554,554,freeport,40.6576,-73.5832,13.43,46,89,2.24,US,1666108554
555,555,bulgan,48.8125,103.5347,5.37,40,67,1.26,MN,1666108554
560,560,changli,39.7042,119.1506,9.17,33,0,2.41,CN,1666108557
565,565,wyndham,37.6985,-77.6125,12.81,52,3,1.79,US,1666108480


### Step 3: Create a new DataFrame called `hotel_df`.

In [29]:
# Use the Pandas copy function to create DataFrame called hotel_df to store the city, country, coordinates, and humidity
hotel_df = criteria_df.copy()
hotel_df = hotel_df[["City", "Country", "Lat", "Lng", "Humidity"]]

# Add an empty column, "Hotel Name," to the DataFrame so you can store the hotel found using the Geoapify API
hotel_df["Hotel Name"] = ""

# Display sample data
hotel_df.head()

,City,Country,Lat,Lng,Humidity,Hotel Name
0,faya,SA,18.3851,42.4509,35,
2,new norfolk,AU,-42.7826,147.0587,58,
4,lanzhou,CN,36.0564,103.7922,48,
6,albany,US,42.6001,-73.9662,74,
9,saint-pierre,RE,-21.3393,55.4781,68,


### Step 4: For each city, use the Geoapify API to find the first hotel located within 10,000 metres of your coordinates.

In [30]:
# Set parameters to search for a hotel
radius = 10000
params = {"categories": "accommodation.hotel",
          "apiKey": geoapify_key,
          "limit": 5
}

# Print a message to follow up the hotel search
print("Starting hotel search")

# Iterate through the hotel_df DataFrame
for index, row in hotel_df.iterrows():
    # get latitude, longitude from the DataFrame
    latitude = row["Lat"]
    longitude = row["Lng"]
    
    # Add filter and bias parameters with the current city's latitude and longitude to the params dictionary
    params["filter"] = f"circle:{longitude},{latitude},{radius}"
    params["bias"] = f"proximity:{longitude},{latitude}"
    
    # Set base URL
    base_url = "https://api.geoapify.com/v2/places"


    # Make and API request using the params dictionaty
    name_address = requests.get(base_url, params=params)
    
    # Convert the API response to JSON format
    name_address = name_address.json()
    
    # Grab the first hotel from the results and store the name in the hotel_df DataFrame
    try:
        hotel_df.loc[index, "Hotel Name"] = name_address["features"][0]["properties"]["name"]
    except (KeyError, IndexError):
        # If no hotel is found, set the hotel name as "No hotel found".
        hotel_df.loc[index, "Hotel Name"] = "No hotel found"
        
    # Log the search results
    print(f"{hotel_df.loc[index, 'City']} - nearest hotel: {hotel_df.loc[index, 'Hotel Name']}")

# Display sample data
hotel_df

Starting hotel search
faya - nearest hotel: No hotel found
new norfolk - nearest hotel: Woodbridge on the Derwent
lanzhou - nearest hotel: 西园宾馆
albany - nearest hotel: No hotel found
saint-pierre - nearest hotel: Tropic Hotel
avarua - nearest hotel: Paradise Inn
constitucion - nearest hotel: Hostal Alameda
bambous virieux - nearest hotel: Otentic Eco Tent Experience
zangakatun - nearest hotel: No hotel found
midwest city - nearest hotel: No hotel found
naberera - nearest hotel: No hotel found
drochia - nearest hotel: Hotel Confort
klaksvik - nearest hotel: Hotel Klaksvik
ancud - nearest hotel: Cabañas
collierville - nearest hotel: Days Inn & Suites by Wyndham Collierville Germantown Area
hobart - nearest hotel: Vibe Hotel Hobart
lasa - nearest hotel: Hotel "Drei Kreuz"
grindavik - nearest hotel: The Retreat at Blue Lagoon Iceland
banda - nearest hotel: #acnindiafy21
zamberk - nearest hotel: Hotel na Zámku
mahebourg - nearest hotel: Grand Bel Air
rafaela - nearest hotel: Parra Hotel & S

,City,Country,Lat,Lng,Humidity,Hotel Name
0,faya,SA,18.3851,42.4509,35,No hotel found
2,new norfolk,AU,-42.7826,147.0587,58,Woodbridge on the Derwent
4,lanzhou,CN,36.0564,103.7922,48,西园宾馆
6,albany,US,42.6001,-73.9662,74,No hotel found
9,saint-pierre,RE,-21.3393,55.4781,68,Tropic Hotel
...,...,...,...,...,...,...
554,freeport,US,40.6576,-73.5832,46,The Freeport Inn and Marina
555,bulgan,MN,48.8125,103.5347,40,Bulgan Hotel
560,changli,CN,39.7042,119.1506,33,客房
565,wyndham,US,37.6985,-77.6125,52,Hilton Richmond Hotel & Spa/Short Pump


### Step 5: Add the hotel name and the country as additional information in the hover message for each city in the map.

In [31]:
%%capture --no-display

# Configure the map plot
hotels_map = hotel_df.hvplot.points(
    "Lng",
    "Lat",
    geo = True,
    tiles = "EsriNatGeo",
    frame_width = 700,
    frame_height = 500,
    size = "Humidity",
    scale = 1,
    color = "City",
    hover_cols = ["Hotel Name", "Country"]
)
# Display the map
hotel_map

:Overlay
   .WMTS.I   :WMTS   [Longitude,Latitude]
   .Points.I :Points   [Lng,Lat]   (City,Humidity,Country,Hotel Name)